In [ ]:
setwd("/home/jfmill5/econ470/a0/work/hwk3/data-build")

In [ ]:
if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, scales)

source("functions.r")
source("rating_variables.r")

monthlist <- sprintf("%02d", 1:12)

# Set the year 2013
y <- 2013

In [ ]:
# plan data
  # Month list --------------------------------------------------------------
  monthlist <- if (y == 2006) sprintf("%02d", 7:12) else sprintf("%02d", 1:12)

  # Readers (quiet & typed) -------------------------------------------------
  read_contract <- function(path) {
    read_csv(
      path,
      skip = 1,
      col_names = c(
        "contractid","planid","org_type","plan_type","partd","snp","eghp",
        "org_name","org_marketing_name","plan_name","parent_org","contract_date"
      ),
      col_types = cols(
        contractid = col_character(),
        planid     = col_double(),
        org_type   = col_character(),
        plan_type  = col_character(),
        partd      = col_character(),
        snp        = col_character(),
        eghp       = col_character(),
        org_name   = col_character(),
        org_marketing_name = col_character(),
        plan_name  = col_character(),
        parent_org = col_character(),
        contract_date = col_character()
      ),
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  read_enroll <- function(path) {
    read_csv(
      path,
      skip = 1,
      col_names = c("contractid","planid","ssa","fips","state","county","enrollment"),
      col_types = cols(
        contractid = col_character(),
        planid     = col_double(),
        ssa        = col_double(),
        fips       = col_double(),
        state      = col_character(),
        county     = col_character(),
        enrollment = col_double()
      ),
      na = "*",
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  # One-month loader --------------------------------------------------------
  load_month <- function(m, y) {
    c_path <- paste0("../../ma-data/ma/enrollment/Extracted Data/CPSC_Contract_Info_", y, "_", m, ".csv") 
    e_path <- paste0("../../ma-data/ma/enrollment/Extracted Data/CPSC_Enrollment_Info_", y, "_", m, ".csv")

    contract.info <- read_contract(c_path) %>%
      distinct(contractid, planid, .keep_all = TRUE)   

    enroll.info <- read_enroll(e_path)

    contract.info %>%
      left_join(enroll.info, by = c("contractid","planid")) %>%
      mutate(month = as.integer(m), year = y)
  }

  # Read all months, then tidy once ----------------------------------------

  plan.year <- map_dfr(monthlist, ~ load_month(.x, y)) %>%
    arrange(contractid, planid, state, county, month) %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%                
    ungroup() %>%
    group_by(contractid, planid) %>%
    fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%
    ungroup() %>%
    group_by(contractid) %>%
    fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%
    ungroup()


  # Collapse to yearly panel ------------------------------------------------
  final.plans <- plan.year %>%
    group_by(contractid, planid, fips, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      n_nonmiss        = sum(!is.na(enrollment)),
      avg_enrollment   = ifelse(n_nonmiss > 0, mean(enrollment, na.rm = TRUE), NA_real_),
      sd_enrollment    = ifelse(n_nonmiss > 1, sd(enrollment, na.rm = TRUE), NA_real_),
      min_enrollment   = ifelse(n_nonmiss > 0, min(enrollment, na.rm = TRUE), NA_real_),
      max_enrollment   = ifelse(n_nonmiss > 0, max(enrollment, na.rm = TRUE), NA_real_),
      first_enrollment = ifelse(n_nonmiss > 0, first(na.omit(enrollment)), NA_real_),
      last_enrollment  = ifelse(n_nonmiss > 0,  last(na.omit(enrollment)), NA_real_),
      state            = last(state),
      county           = last(county),
      org_type         = last(org_type),
      plan_type        = last(plan_type),
      partd            = last(partd),
      snp              = last(snp),
      eghp             = last(eghp),
      org_name         = last(org_name),
      org_marketing_name = last(org_marketing_name),
      plan_name        = last(plan_name),
      parent_org       = last(parent_org),
      contract_date    = last(contract_date),
      year             = last(year),
      .groups = "drop"
    )

In [ ]:
# service area 
  # Month list --------------------------------------------------------------
  monthlist <- if (y == 2006) sprintf("%02d", 10:12) else sprintf("%02d", 1:12)

  # Reader ------------------------------------------------------------------
  read_service_area <- function(path) {
    read_csv(
      path, skip = 1,
      col_names = c(
        "contractid","org_name","org_type","plan_type","partial","eghp",
        "ssa","fips","county","state","notes"
      ),
      col_types = cols(
        contractid = col_character(),
        org_name   = col_character(),
        org_type   = col_character(),
        plan_type  = col_character(),
        partial    = col_logical(),
        eghp       = col_character(),
        ssa        = col_double(),
        fips       = col_double(),
        county     = col_character(),
        state      = col_character(),
        notes      = col_character()
      ),
      na = "*",
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  # One-month loader --------------------------------------------------------
  load_month_sa <- function(m, y) {
    path <- paste0("../../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_",y, "_", m, ".csv")
    
    read_service_area(path) %>%
      mutate(month = as.integer(m), year = y)
  }

  # Read all months, then tidy once ----------------------------------------
  service.year <- map_dfr(monthlist, ~ load_month_sa(.x, y))

  # Ensure stable order before fills
  service.year <- service.year %>%
    arrange(contractid, fips, state, county, month)

  # Fill missing identifiers/labels
  service.year <- service.year %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%
    ungroup() %>%
    group_by(contractid) %>%
    fill(plan_type, partial, eghp, org_type, org_name, .direction = "downup") %>%
    ungroup()

  # Collapse to yearly: one row per contract × county (fips) × year --------
  final.service.area <- service.year %>%
    group_by(contractid, fips, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      state     = last(state),
      county    = last(county),
      org_name  = last(org_name),
      org_type  = last(org_type),
      plan_type = last(plan_type),
      partial   = last(partial),
      eghp      = last(eghp),
      ssa       = last(ssa),
      notes     = last(notes),
      .groups = "drop"
    )

In [ ]:
# plan characteristics

# Import data -------------------------------------------------------------

ma.path.a <- paste0("../../ma-data/ma/landscape/Extracted Data/2013LandscapeSource file MA_AtoM 11212012.csv")
ma.data.a <- read_csv(ma.path.a,
                      skip=6,
                      col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                  "drug_type","gap_coverage","drug_type_detail","contractid",
                                  "planid","segmentid","moop","star_rating"),
                      col_types = cols(
                        state = col_character(),
                        county = col_character(),
                        org_name = col_character(),
                        plan_name = col_character(),
                        plan_type = col_character(),
                        premium = col_number(),
                        partd_deductible = col_number(),
                        drug_type = col_character(),
                        gap_coverage = col_character(),
                        drug_type_detail = col_character(),
                        contractid = col_character(),
                        planid = col_double(),
                        segmentid = col_double(),
                        moop = col_character(),
                        star_rating = col_character()
                      ))


ma.path.b <- paste0("../../ma-data/ma/landscape/Extracted Data/2013LandscapeSource file MA_NtoW 11212012.csv")
ma.data.b <- read_csv(ma.path.b,
                      skip=6,
                      col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                  "drug_type","gap_coverage","drug_type_detail","contractid",
                                  "planid","segmentid","moop","star_rating"),
                      col_types = cols(
                        state = col_character(),
                        county = col_character(),
                        org_name = col_character(),
                        plan_name = col_character(),
                        plan_type = col_character(),
                        premium = col_number(),
                        partd_deductible = col_number(),
                        drug_type = col_character(),
                        gap_coverage = col_character(),
                        drug_type_detail = col_character(),
                        contractid = col_character(),
                        planid = col_double(),
                        segmentid = col_double(),
                        moop = col_character(),
                        star_rating = col_character()
                      ))

ma.data <- rbind(ma.data.a,ma.data.b)


mapd.path.a <- paste0("../../ma-data/ma/landscape/Extracted Data/PartCD/2013/Medicare Part D 2013 Plan Report 04252013v1.xls")
mapd.data.a <- read_xls(mapd.path.a,
                        range="A5:AA20940",
                        sheet="Alabama to Montana",
                        col_names=c("state","county","org_name","plan_name","contractid","planid","segmentid",
                                    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
                                    "national_pdp","premium_partc",
                                    "premium_partd_basic","premium_partd_supp","premium_partd_total",
                                    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
                                    "partd_deductible","deductible_exclusions","increase_coverage_limit",
                                    "gap_coverage","gap_coverage_type"))



mapd.path.b <- paste0("../../ma-data/ma/landscape/Extracted Data/PartCD/2013/Medicare Part D 2013 Plan Report 04252013v1.xls")
mapd.data.b <- read_xls(mapd.path.b,
                        range="A5:AA23812",
                        sheet="Nebraska to Wyoming",
                        col_names=c("state","county","org_name","plan_name","contractid","planid","segmentid",
                                    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
                                    "national_pdp","premium_partc",
                                    "premium_partd_basic","premium_partd_supp","premium_partd_total",
                                    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
                                    "partd_deductible","deductible_exclusions","increase_coverage_limit",
                                    "gap_coverage","gap_coverage_type"))
mapd.data <- rbind(mapd.data.a,mapd.data.b)

final.landscape <- mapd.clean.merge(ma.data=ma.data, mapd.data=mapd.data, y)


In [ ]:
# penetration
# penetration
  # Month list --------------------------------------------------------------
  monthlist <- if (y == 2008) sprintf("%02d", 6:12) else sprintf("%02d", 1:12)

  # Reader: read numerics as text, then parse --------------------------------
  read_penetration <- function(path) {
    raw <- read_csv(
      path,
      skip = 1,
      col_names = c(
        "state","county","fips_state","fips_cnty","fips",
        "ssa_state","ssa_cnty","ssa","eligibles","enrolled","penetration"
      ),
      # read potential problem columns as character first
      col_types = cols(
        state      = col_character(),
        county     = col_character(),
        fips_state = col_integer(),
        fips_cnty  = col_integer(),
        fips       = col_double(),
        ssa_state  = col_integer(),
        ssa_cnty   = col_integer(),
        ssa        = col_double(),
        eligibles  = col_character(),
        enrolled   = col_character(),
        penetration= col_character()
      ),
      na = c("", "NA", "*", "-", "--"),
      show_col_types = FALSE,
      progress = FALSE
    )

    # robust numeric parsing (handles commas, %, stray text)
    raw %>%
      mutate(
        eligibles   = parse_number(eligibles),
        enrolled    = parse_number(enrolled),
        penetration = parse_number(penetration)
      )
  }

  # One-month loader --------------------------------------------------------
  load_month_pen <- function(m, y) {
    path <- paste0("../../ma-data/ma/penetration/Extracted Data/State_County_Penetration_MA_",y, "_", m, ".csv")

    read_penetration(path) %>%
      mutate(month = as.integer(m), year = y)
  }

  # Read all months, then tidy once ----------------------------------------
  ma.penetration <- map_dfr(monthlist, ~ load_month_pen(.x, y)) %>%
    arrange(state, county, month) %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%
    ungroup()

  # Collapse to yearly (safe summaries; avoid NaN/Inf) ----------------------
  final.penetration <- ma.penetration %>%
    group_by(fips, state, county, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      n_elig  = sum(!is.na(eligibles)),
      n_enrol = sum(!is.na(enrolled)),

      avg_eligibles   = ifelse(n_elig  > 0, mean(eligibles, na.rm = TRUE), NA_real_),
      sd_eligibles    = ifelse(n_elig  > 1,  sd(eligibles,  na.rm = TRUE), NA_real_),
      min_eligibles   = ifelse(n_elig  > 0, min(eligibles,  na.rm = TRUE), NA_real_),
      max_eligibles   = ifelse(n_elig  > 0, max(eligibles,  na.rm = TRUE), NA_real_),
      first_eligibles = ifelse(n_elig  > 0, first(na.omit(eligibles)),     NA_real_),
      last_eligibles  = ifelse(n_elig  > 0,  last(na.omit(eligibles)),     NA_real_),

      avg_enrolled    = ifelse(n_enrol > 0, mean(enrolled,   na.rm = TRUE), NA_real_),
      sd_enrolled     = ifelse(n_enrol > 1,  sd(enrolled,    na.rm = TRUE), NA_real_),
      min_enrolled    = ifelse(n_enrol > 0, min(enrolled,    na.rm = TRUE), NA_real_),
      max_enrolled    = ifelse(n_enrol > 0, max(enrolled,    na.rm = TRUE), NA_real_),
      first_enrolled  = ifelse(n_enrol > 0, first(na.omit(enrolled)),       NA_real_),
      last_enrolled   = ifelse(n_enrol > 0,  last(na.omit(enrolled)),       NA_real_),

      ssa = last(ssa),
      .groups = "drop"
    )

In [ ]:
# star ratings

# Import data -------------------------------------------------------------

ma.path.a <- "../../ma-data/ma/star-ratings/Extracted Star Ratings/Part C 2013 Fall/2013_Part_C_Report_Card_Master_Table_2012_10_17_Star.csv"
star.data.a <- read_csv(
  ma.path.a,
  skip = 4,
  col_names = rating.vars.2013,
  na = c("", "NA", "*")
) %>%
  mutate(across(
    -any_of(c("contractid","org_type","contract_name","org_marketing","org_parent")),
    ~ parse_number(as.character(.))
  ))

ma.path.b <- "../../ma-data/ma/star-ratings/Extracted Star Ratings/Part C 2013 Fall/2013_Part_C_Report_Card_Master_Table_2012_10_17_Summary.csv"
star.data.b <- read_csv(
  ma.path.b,
  skip = 2,
  col_names=c("contractid","org_type","org_marketing","contract_name","org_parent","partc_score","partc_lowscore","partc_highscore",
              "partcd_score","partcd_lowscore","partcd_highscore"),
  na = c("", "NA", "*")
) %>%
  mutate(
    new_contract=ifelse(partc_score=="Plan too new to be measured",1, ifelse(partcd_score=="Plan too new to be measured",1,0)),
    partc_score  = ifelse(new_contract == 1, NA_real_, parse_number(as.character(partc_score))),
    partcd_score = ifelse(new_contract == 1, NA_real_, parse_number(as.character(partcd_score))),
    low_score=as.numeric(ifelse(partc_lowscore=="Yes",1,0))
  ) %>%
  select(contractid, new_contract, low_score, partc_score, partcd_score)

final.star.ratings <- star.data.a %>%
  select(-contract_name, -org_type, -org_marketing) %>%  
  left_join(star.data.b, by=c("contractid")) %>%
  mutate(year=2013)

In [ ]:
# Merge data
ma.2013 <- final.plans %>% 
  inner_join(final.service.area %>% select(contractid, fips, ssa), by = c("contractid", "fips")) %>% 
  filter(!state %in% c("VI", "PR", "MP", "GU", "AS", ""), 
         snp == "No", 
         (planid < 800 | planid >= 900), 
         !is.na(planid), 
         !is.na(fips))

ma.2013 <- ma.2013 %>% 
  left_join(final.penetration %>% 
              ungroup() %>% 
              rename(state_long = state, county_long = county) %>% 
              mutate(state_long = str_to_lower(state_long)) %>% 
              group_by(fips) %>% 
              filter(n() == 1) %>%
              select(fips, state_long, county_long, avg_eligibles, avg_enrolled), 
            by = "fips")

state.2013 <- ma.2013 %>% 
  group_by(state) %>% 
  summarize(state_name = last(state_long[!is.na(state_long)]), .groups = "drop")

full.2013 <- ma.2013 %>% 
  left_join(state.2013, by = "state") %>% 
  left_join(landscape.2013 %>% 
              mutate(state = str_to_lower(state)) %>%
              select(contractid, planid, state, county, premium, partd_deductible), 
            by = c("contractid", "planid", "state_name" = "state", "county")) %>% 
  left_join(final.star.ratings %>%
              select(contractid, partc_score, partcd_score, new_contract), 
            by = "contractid") %>% 
  mutate(year = 2013)

In [ ]:
write_csv(full.2013, "../data/data-2013.csv")

In [ ]:
names(full.2013)